In [1]:
from typing import Any
from dataclasses import dataclass
import torch
import torch.nn as nn  # NN networks (CNN, RNN, losses)
import torch.optim as optim  # Aptimizers (Adam, Adadelta, Adagrad)
# Activarions func (ReLU, Sigmoid) also included in nn
import torch.nn.functional as F
from torch.utils.data import DataLoader  # Dataset manager
import torchvision.datasets as datasets  # Datasets
import torchvision.transforms as transforms  # Transformation to datasets
# import torchvision

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = "cuda" if torch.cuda.is_available() else "cpu"


In [2]:
# Convs
VGG16_list = [64, 64, "M", 128, 128, "M",
              256, 256, 256, "M", 512, 512, 512, "M", 512, 512, 512, "M"]


In [3]:

class VGG16(nn.Module):
    def __init__(self, in_channels=3, num_classes=1000) -> None:
        super().__init__()
        self.in_channels = in_channels
        self.num_classes = num_classes
        self.conv_layers = self.create_conv_layers(VGG16_list)
        
        self.fcs = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(4096, self.num_classes)
        )

    def forward(self, x: Any):
        x = self.conv_layers(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fcs(x)
        return x

    def create_conv_layers(self, architecture):
        layers = []
        in_channels = self.in_channels
        for x in architecture:
            if type(x) == int:
                out_channels = x

                layers += [nn.Conv2d(in_channels=in_channels,
                                     out_channels=out_channels,
                                     kernel_size=(3, 3),
                                     stride=(1, 1),
                                     padding=(1, 1)
                                     ),
                           # nn.BatchNorm2d(x),
                           nn.ReLU()]
                in_channels = x
            elif x == "M":
                layers.append(nn.MaxPool2d(kernel_size=(2, 2),
                                           stride=(2, 2)))
        return nn.Sequential(*layers)



In [4]:
model = VGG16()
model.to(device=device)
x = torch.randn(1, 3, 224, 224)
print(model(x).shape)

RuntimeError: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same or input should be a MKLDNN tensor and weight is a dense tensor